# import library

In [351]:
import numpy as np
import pandas as pd
from utils import *

In [ ]:
from scipy.cluster.vq import kmeans2

In [10]:
from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import linkage,dendrogram,distance 
from sklearn.metrics import calinski_harabasz_score,fowlkes_mallows_score,silhouette_score

In [2]:
from matplotlib import pyplot as plt
from matplotlib import cm
from IPython.core.pylabtools import figsize
%matplotlib widget

In [94]:
ac = AgglomerativeClustering(n_clusters=5,)

In [358]:
pred = ac.fit_predict(v_data)

'braycurtis', 'canberra', 'chebyshev', 'cityblock','correlation', 'cosine', 'dice', 'euclidean', 'hamming',
    'jaccard', 'jensenshannon', 'kulsinski', 'mahalanobis', 'matching',
    'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean',
    'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule'

In [37]:
disMat=distance.pdist(v_data,metric='euclidean')

In [38]:
Z=linkage(disMat,method='average') 

In [40]:
plt.figure()
tree = dendrogram(Z)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [350]:
ac = AgglomerativeClustering(n_clusters=5)
data = np.random.random((3149,22))
pred = ac.fit_predict(data) 

In [483]:
label_ls = [AgglomerativeClustering(n_clusters=i).fit_predict(v_data) for i in range(5,20)]

# metrics to evaulate 

### Calinski-Harabasz index
manunally compute SS_b and SS_w

$$ CH = \frac{SS_B}{SS_W} \times \frac{N-k}{k-1} $$

In [24]:
a=np.ones((2,4))

In [51]:
import my_metrics as mm

In [53]:
from importlib import reload

In [66]:
reload(mm)

<module 'my_metrics' from '/home/ZwZ/script/HER2_prediction/my_metrics.py'>

### silhouette coefficience

$$S_i = \frac{\beta_i - \alpha_i}{\max(\alpha_i,\beta_i)}$$

In [250]:
# %load -s my_silhouette my_metrics.py
def my_silhouette(X,label,metric='euclidean'):
    r"""math:`S_i = \frac{\beta_i - \alpha_i}{\max(\alpha_i,\beta_i)}`
    """
    
    S = np.zeros(X.shape[0])
    alpha = np.zeros(X.shape[0])
    beta = np.zeros(X.shape[0])
    cluster_data = {}
    for cluster in np.unique(label):
         cluster_data[cluster] = X[label == cluster]            # ie. (100,22)
        
    for cluster in np.unique(label):
        c_index = np.where(label == cluster)[0]
        # ----------- compute a_i of the current class ------------
        X_c = cluster_data[cluster]
        alpha_dist = pairwise.pairwise_distances(X_c)
        #alpha = np.mean(alpha_dist,axis=1)
        alpha[c_index] = np.sum(alpha_dist,axis=1)/(alpha_dist.shape[1]-1)
        
        # ----------- compute b_i of the current class ------------        
        beta_ls = []
        for other in np.unique(label):
            if cluster == other:
                continue
            X_nc = cluster_data[other]                           # ie. (300,22)
            beta_dist = pairwise.pairwise_distances(X_c,X_nc)    # ie. (100,300)
            beta_ls.append(np.mean(beta_dist,axis=1))
        beta[c_index] = np.min(np.stack(beta_ls),axis=0)
        
        #x_nc = X[label != cluster]
        #beta_dist = pairwise.pairwise_distances(X_c,X_nc)
        #beta = np.mean(beta_dist,axis=1)
        
        # ----------- compute S_i of the current class ------------
                
        S[c_index] = (beta[c_index] - alpha[c_index])/np.maximum(beta[c_index],alpha[c_index])

    return S


In [252]:
my_silhouette(v_dat.)

In [240]:
sil_samples,intra_clust_dists,inter_clust_dists = silhouette_samples(v_data,pred)

### Jump score
$$d_k = \frac{1}{p} \min_{c_1,...,c_k} E[(X-C_x)^T\Sigma^{-1}(X-C_x)]$$
$$J_k = d_k-d_{k-1} $$

In [228]:
def jump_of_k(X,labels):
    p = X.shape[1]
    Centoid = np.stack([np.mean(X[labels == cluster],axis=0) for cluster in np.unique(labels)])
    dist_M = pairwise.pairwise_distances(X,Centoid,metric='mahalanobis')
    dk =  np.min(np.mean(dist_M,axis=0))/p
    return dk

### Gap statistics

$$ Gap(k)=\frac{1}{B} \sum_{b=1}^B \log(W_{kb}^*) - \log(W_k) $$ 

*note: the cluster center of random datas should be determined by itself with k-means

$$ w' = \frac{1}{B} \sum_{b=1}^B \log(W_{kb}^*) $$

$$ sd(k) = \sqrt{ \frac{1}{B} \sum_{b} (\log(W_{kb}^*) - w')^2} $$

$$ s_k = \sqrt{\frac{1+B}{B} sd(k)} $$

$$ k = \min k $$
$$s.t.   Gap(k) >= Gap(k+1) +s_{k+1}$$

In [473]:
def gap_statistics(X,labels,n_random=100):
    r""" compute the Gap(k) and sk for current data with label
    refer: 
    ... https://blog.csdn.net/baidu_17640849/article/details/70769555?fps=1&locationNum=2
    """
    n_cluster = len(np.unique(labels))
    
    # compute Wk
    Wk =  compute_wk(X,labels)
    
    # random sample and compute Wkb
    W_prims = []
    data_range = np.matrix(np.diag(X.max(axis=0) - X.min(axis=0)))

    for r in range(n_random):
        refs = np.random.random(X.shape)*data_range + X.min(axis=0)    # ie. (3194,22)
        refs_center,refs_label = kmeans2(refs,n_cluster)        # ie. (5,22) ,(3194,)
        refs_center = refs_center.take(refs_label,axis=0)              # ie. (3194,22)
        Wkb = pairwise.paired_distances(refs,refs_center)    #  sum ( (3194,) ) -> R
        assert(Wkb.shape == (X.shape[0],))
        W_prims.append(np.log(np.sum(Wkb)))
    
    W_prim = np.mean(W_prims)
    
    # compute sd_k
    sd_k = np.sqrt(np.mean((np.array(W_prims) - W_prim)**2))
    s_k = np.sqrt((1+n_random)*sd_k/n_random )
    
    # compute Gap(k)
    Gap_k = W_prim - np.log(Wk)
    return Gap_k , s_k

In [427]:
gap_statistics(v_data,pred)

(1.0323663849298308, 0.055648033901140696)

In [488]:
import scipy
import scipy.cluster.vq
import scipy.spatial.distance
import numpy as np
EuclDist = scipy.spatial.distance.euclidean
def gapStat(data, resf=None, nrefs=10, ks=range(1,10)):
    '''
    Gap statistics
    '''
    # MC
    shape = data.shape
    if resf == None:
        x_max = data.max(axis=0)
        x_min = data.min(axis=0)
        dists = np.matrix(np.diag(x_max-x_min))
        rands = np.random.random_sample(size=(shape[0], shape[1], nrefs))
        for i in range(nrefs):
            rands[:,:,i] = rands[:,:,i]*dists+x_min
    else:
        rands = refs
    gaps = np.zeros((len(ks),))
    gapDiff = np.zeros(len(ks)-1,)
    sdk = np.zeros(len(ks),)
    for (i,k) in enumerate(ks):
        (cluster_mean, cluster_res) = scipy.cluster.vq.kmeans2(data, k)
        Wk = sum([EuclDist(data[m,:], cluster_mean[cluster_res[m],:]) for m in range(shape[0])])
        WkRef = np.zeros((rands.shape[2],))
        for j in range(rands.shape[2]):
            (kmc,kml) = scipy.cluster.vq.kmeans2(rands[:,:,j], k)
            WkRef[j] = sum([EuclDist(rands[m,:,j],kmc[kml[m],:]) for m in range(shape[0])])
        gaps[i] = scipy.log(scipy.mean(WkRef))-scipy.log(Wk)
        sdk[i] = np.sqrt((1.0+nrefs)/nrefs)*np.std(scipy.log(WkRef))

        if i > 0:
            gapDiff[i-1] = gaps[i-1] - gaps[i] + sdk[i]
    return gaps, gapDiff

In [489]:
gapStat(v_data)

/home/ZwZ/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: scipy.mean is deprecated and will be removed in SciPy 2.0.0, use numpy.mean instead
/home/ZwZ/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: scipy.log is deprecated and will be removed in SciPy 2.0.0, use numpy.lib.scimath.log instead
/home/ZwZ/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: scipy.log is deprecated and will be removed in SciPy 2.0.0, use numpy.lib.scimath.log instead


(array([0.89662957, 0.96167973, 1.02684054, 1.0582979 , 1.07790736,
        1.08962838, 1.10040198, 1.12412361, 1.1318255 ]),
 array([-0.06343369, -0.06293012, -0.02724294, -0.01728405, -0.00974109,
        -0.0084144 , -0.02164338, -0.00517767]))

## DB method
<font size=4>
$$ DB = \frac{1}{k} \sum_{k=1}^k R_i$$
$$ R_i = \max_{j=1...k,i \neq j} R_{ij}$$
$$ R_{ij} = \frac{s_i + s_j}{d_{ij}} $$
$$ d_{ij} = d(v_i,v_j)$$</font>
where $v_i$ is the cluster center for class $i$  
<font size=4>
$$ s_i = \frac{1}{||c_i||} \sum_{x \in c_i} d(x,v_i)$$ </font>
*note $||c_i||$ is scale by the size of class $i$

In [459]:
def DB_score(X,labels):
    
    n_cluster = len(np.unique(labels))
    # get v
    v = np.stack([np.mean(X[labels==c],axis=0) for c in np.unique(labels)])

    # d_ij
    d_ij = pairwise.pairwise_distances(v)
    for i in range(d_ij.shape[0]):
        d_ij[i,i] = np.inf

    # compute si
    s_i = np.stack([np.mean(pairwise.pairwise_distances(X[labels ==c],v[c].reshape(1,-1))) for c in np.unique(labels)])
    s_i_bc = np.broadcast_to(s_i,(n_cluster,n_cluster))

    # compute R
    R_ij = np.divide((s_i_bc.T + s_i_bc),d_ij)
    R_i = np.max(R_ij,axis=1)

    # compute DB
    DB = np.mean(R_i)

    return DB

##  Krzanowski & Lai 
$$ DIFF(k) = (k-1)^{2/p}W_{k-1} - k^{2/p}W_k$$  
&nbsp;
$$ KL(k) = |\frac{DIFF(k)}{DIFF(k+1} |$$

In [495]:
def compute_wk(X,labels,KL=False):
    """
    Wk stands for the sum of euclidean distance of data points to its cooresponding cluster center
    arguments:
    ...X : (n_samples,n_feature)
    ...labels : (n_samples,) dtype = int
    ...KL : recale by k and data dimension p , should set to 'True' when later used to 
    """
    p = X.shape[1]
    n_cluster = len(np.unique(labels))
    # get v
    v = np.stack([np.mean(X[labels==c],axis=0) for c in np.unique(labels)])

    # compute si
    wk = np.sum([np.sum(pairwise.pairwise_distances(X[labels ==c],v[c].reshape(1,-1))) for c in np.unique(labels)])
    ## 
    if KL:
        wk = (n_cluster)**(2/p)*wk
    
    return wk

def KL_score(X,label_ls):
    '''
    when given data X and a list containing cluster labels with different n_clusters setting
    argument:
    ...X : (n_samples,n_feature)
    ...label_ls : list 
    outpur:
    ... KL_k : list , length : len(label_ls) - 2
    '''
    k_ls = [len(np.unique(labels)) for labels in label_ls]
    wk_ls = [compute_wk(X,labels,KL=True) for labels in label_ls]
    
    # len : ks -1
    DIFF_k = [wk_ls[i-1] - wk_ls[i] for i in range(1,len(label_ls))]
    #
    KL_k = [np.abs(np.divide(DIFF_k[i],DIFF_k[i+1])) for i in range(0,len(DIFF_k)-1)]
    return KL_k

In [496]:
KL_score(v_data,label_ls)

[1.3758308621657853,
 0.6831266543112948,
 0.740430633655721,
 1.199114559438958,
 2.7226158423891142,
 1.1037197644914154,
 0.8279054265084943,
 0.8183576685961351,
 0.8906785997216118,
 1.1487437766258237,
 1.2454179777800887,
 1.242924589671839,
 1.6712855287659611]

## Hartigan method
$$H(k) = (\frac{W_k}{W_{k+1}} -1)(n-k-1)$$

In [500]:
def Hartigan_score(X,label_ls):
    """
    the calculation of Hartigan score  require Wk and Wk+1 
    """
    n = X.shape[0]
    k_ls = [len(np.unique(labels)) for labels in label_ls]
    
    # remember to set KL=True , for Wk share the same definition with that in KL method
    wk_ls = [compute_wk(X,labels,KL=True) for labels in label_ls]
    
    # compute Hk , require W_{k} and W_{k+1}
    Hk_ls = [(wk_ls[i]/wk_ls[i+1] -1)*(n-k_ls[i]-1) for i in range(0,len(k_ls)-1)]
    
    return Hk_ls

In [501]:
Hartigan_score(v_data,label_ls)

[24.347179913744952,
 17.790886085023395,
 26.25263334590904,
 35.84928033494645,
 30.174232593246618,
 11.11853477665086,
 10.10291607509216,
 12.246733260964845,
 15.031973810180697,
 16.96292495616799,
 14.831670806761027,
 11.950615340324068,
 9.641444191517095,
 5.777682017484094]

## ALL in one combat

In [534]:
reload(my_metrics)

from my_metrics import *

In [535]:
metrics = cluster_metrics(v_data,label_ls)